In [ ]:
# Cell 1: Import necessary libraries (No change)
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd
from collections import defaultdict

In [ ]:
# Cell 2: Function to roll a dice (No change)
def roll_dice():
    """Simulates a single six-sided dice roll."""
    return random.randint(1, 6)

In [ ]:
# Cell 3:  Modified create_board function with Targeted Debugging in generate_endpoints and _check_overlap
def create_board(board_size, num_snakes, num_ladders, snake_length=10, ladder_length=10):
    """
    Generates random positions for snakes and ladders with FIXED lengths - DEBUGGING OVERLAP.
    With targeted print statements in generate_endpoints and _check_overlap.
    """
    snakes = {}
    ladders = {}
    used_tiles = set()

    def generate_endpoints(entity_type, used_tiles, board_size, fixed_snake_length, fixed_ladder_length):
        attempts = 0
        while attempts < 100: # Reduced attempts for faster debugging - can increase later
            attempts += 1
            print(f"  generate_endpoints attempt {attempts} for {entity_type}") # DEBUG PRINT
            if entity_type == "snake":
                start_tile = random.randint(snake_length + 2, board_size * board_size - 1)
                end_tile = start_tile - fixed_snake_length
            elif entity_type == "ladder":
                start_tile = random.randint(2, board_size * board_size - fixed_ladder_length - 1)
                end_tile = start_tile + fixed_ladder_length

            if start_tile in used_tiles:
                print(f"    {entity_type} start_tile {start_tile} in used_tiles - continue") # DEBUG PRINT
                continue

            if entity_type == "snake":
                if end_tile < 1:
                    print(f"    {entity_type} end_tile {end_tile} < 1 - continue") # DEBUG PRINT
                    continue
            elif entity_type == "ladder":
                if end_tile > board_size * board_size:
                    print(f"    {entity_type} end_tile {end_tile} > board_size*board_size - continue") # DEBUG PRINT
                    continue

            if end_tile in used_tiles or end_tile == start_tile:
                print(f"    {entity_type} end_tile {end_tile} in used_tiles or == start_tile - continue") # DEBUG PRINT
                continue

            valid_placement = _check_overlap(snakes, ladders, start_tile, end_tile, entity_type)

            if valid_placement:
                used_tiles.add(start_tile)
                used_tiles.add(end_tile)
                print(f"    {entity_type} placement VALID - start: {start_tile}, end: {end_tile}") # DEBUG PRINT
                return start_tile, end_tile
            else:
                print(f"    {entity_type} placement INVALID - overlap check failed") # DEBUG PRINT

        print(f"  generate_endpoints FAILED after {attempts} attempts for {entity_type}") # DEBUG PRINT
        return None, None

    def _check_overlap(snakes, ladders, start_tile, end_tile, entity_type):
        """Helper function to check for overlaps - WITH DEBUG PRINT."""
        print(f"    _check_overlap called for {entity_type} start: {start_tile}, end: {end_tile}") # DEBUG PRINT
        valid_placement = True
        for s_start, s_end in snakes.items():
            if start_tile == s_start or start_tile == s_end or end_tile == s_start or end_tile == s_end:
                valid_placement = False
                print(f"      Overlap with snake at start/end: snake_start={s_start}, snake_end={s_end}") # DEBUG PRINT
                break
        if not valid_placement:
            return False

        for l_start, l_end in ladders.items():
            if start_tile == l_start or start_tile == l_end or end_tile == l_start or end_tile == l_end:
                valid_placement = False
                print(f"      Overlap with ladder at start/end: ladder_start={l_start}, ladder_end={l_end}") # DEBUG PRINT
                break
        print(f"    _check_overlap returning {valid_placement}") # DEBUG PRINT
        return valid_placement


    # Generate Snakes - with retry mechanism if placement fails
    snakes_placed = 0
    for _ in range(num_snakes * 2):
        if snakes_placed >= num_snakes:
            break
        print(f"Attempting to place snake {snakes_placed + 1}/{num_snakes}") # DEBUG PRINT - Snake placement attempt
        start, end = generate_endpoints("snake", used_tiles, board_size * board_size, snake_length, ladder_length)
        if start is not None:
            snakes[start] = end
            snakes_placed += 1
            print(f"Snake placed successfully. Snakes placed: {snakes_placed}/{num_snakes}") # DEBUG PRINT - Snake placed

    # Generate Ladders - with retry mechanism
    ladders_placed = 0
    for _ in range(num_ladders * 2):
        if ladders_placed >= num_ladders:
            break
        print(f"Attempting to place ladder {ladders_placed + 1}/{num_ladders}") # DEBUG PRINT - Ladder placement attempt
        start, end = generate_endpoints("ladder", used_tiles, board_size * board_size, ladder_length, ladder_length)
        if start is not None:
            ladders[start] = end
            ladders_placed += 1
            print(f"Ladder placed successfully. Ladders placed: {ladders_placed}/{num_ladders}") # DEBUG PRINT - Ladder placed

    return snakes, ladders

In [ ]:
# Cell 4: Modified simulate_game function from your provided code (Replaces previous Cell 4)
def simulate_game(board_size, board_snakes, board_ladders): # Renamed snakes and ladders to board_snakes and board_ladders for consistency
    """
    Simulates a single game of Snakes and Ladders.
    Adapted from user-provided code to use board_snakes and board_ladders.
    """
    position = 1 # Start at tile 1
    turns = 0
    target_tile = board_size * board_size # Target tile is board_size * board_size

    while position < target_tile: # Winning condition is reaching board_size*board_size
        roll = roll_dice()
        turns += 1
        new_position = position + roll

        if new_position > target_tile: # Handle overshoot
            position = target_tile # Land exactly on or overshoot to win
        else:
            position = new_position


        if position in board_snakes: # Use board_snakes
            position = board_snakes[position]
        elif position in board_ladders: # Use board_ladders
            position = board_ladders[position]

    return turns

In [ ]:
# Cell 5: Function to run multiple simulations and collect metrics (No Change - but ensure it uses the new simulate_game and create_board functions)
def run_simulations(board_size, num_simulations, num_snakes, num_ladders, snake_length, ladder_length): # snake_length, ladder_length added to args
    """Runs multiple simulations for a given board configuration.

    Args:
        board_size (int): Linear dimension of the board.
        num_simulations (int): Number of simulations to run.
        num_snakes (int): Number of snakes.
        num_ladders (int): Number of ladders.
        snake_length (int): Length of snakes. # Added
        ladder_length (int): Length of ladders. # Added

    Returns:
        dict: Dictionary containing simulation metrics (average_turns, win_probabilities).
    """
    all_turns = []
    for _ in range(num_simulations):
        board_snakes, board_ladders = create_board(board_size, num_snakes, num_ladders, snake_length, ladder_length) # Pass lengths to create_board
        if board_snakes is None: # Handle board generation failure (though unlikely with retries in create_board)
            continue # Skip this simulation if board couldn't be created
        turns = simulate_game(board_size, board_snakes, board_ladders)
        all_turns.append(turns)

    average_turns = np.mean(all_turns)
    prob_win_half_turns = np.mean(np.array(all_turns) <= (board_size*board_size) / 2)
    prob_win_third_turns = np.mean(np.array(all_turns) <= (board_size*board_size) / 3)
    prob_win_quarter_turns = np.mean(np.array(all_turns) <= (board_size*board_size) / 4)

    return {
        "average_turns": average_turns,
        "prob_win_half": prob_win_half_turns,
        "prob_win_third": prob_win_third_turns,
        "prob_win_quarter": prob_win_quarter_turns,
        "all_turns": all_turns # Return all turns for error bar calculation and histograms
    }

In [ ]:
# Cell 6: Function to generate and save board layout visual (No Change - but ensure it works with the new board dictionaries)
def generate_board_layout_visual(board_size, board_snakes, board_ladders, filename="board_layout.png"):
    """Generates and saves a visual representation of the board layout.

    Args:
        board_size (int): Linear dimension of the board.
        board_snakes (dict): Snake positions.
        board_ladders (dict): Ladder positions.
        filename (str, optional): Filename to save the image. Defaults to "board_layout.png".
    """
    tiles = board_size * board_size
    fig, ax = plt.subplots(figsize=(board_size, board_size))
    ax.set_xlim(0, board_size)
    ax.set_ylim(0, board_size)
    ax.set_xticks(np.arange(0, board_size + 1), minor=False)
    ax.set_yticks(np.arange(0, board_size + 1), minor=False)
    ax.grid(which='major', color='grey', linestyle='-', linewidth=0.5)
    ax.set_aspect('equal') # Ensure square tiles

    # Remove axis labels and ticks for cleaner visual
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Draw tile numbers
    for tile in range(1, tiles + 1):
        x = (tile - 1) % board_size + 0.5 # x-coordinate of tile centre
        y = board_size - ((tile - 1) // board_size) - 0.5 # y-coordinate of tile centre (counting from top)
        ax.text(x, y, str(tile), ha='center', va='center', fontsize=8)

    # Draw ladders (green)
    for start_tile, end_tile in board_ladders.items():
        start_x = (start_tile - 1) % board_size + 0.5
        start_y = board_size - ((start_tile - 1) // board_size) - 0.5
        end_x = (end_tile - 1) % board_size + 0.5
        end_y = board_size - ((end_tile - 1) // board_size) - 0.5
        ax.arrow(start_x, start_y, end_x - start_x, end_y - start_y,
                 head_width=0.2, head_length=0.2, fc='green', ec='green', linewidth=2, length_includes_head=True)


    # Draw snakes (red)
    for head_tile, tail_tile in board_snakes.items():
        start_x = (head_tile - 1) % board_size + 0.5
        start_y = board_size - ((head_tile - 1) // board_size) - 0.5
        end_x = (tail_tile - 1) % board_size + 0.5
        end_y = board_size - ((end_tile - 1) // board_size) - 0.5
        ax.arrow(start_x, start_y, end_x - start_x, end_y - end_y,
                 head_width=0.2, head_length=0.2, fc='red', ec='red', linewidth=2, length_includes_head=True)


    plt.title("Snakes and Ladders Board Layout")
    plt.savefig(filename) # Save the figure
    plt.close(fig) # Close figure to free memory
    print(f"Board layout image saved to {filename}")

In [ ]:
# Cell 7: Simulation parameters and board sizes (No change)
board_sizes = range(8, 21, 2) # Board sizes from 8x8 to 20x20 in steps of 2
num_simulations = 10000
snake_ladder_length = 10 # Fixed snake and ladder length

# Ratios for Ns/Nl variation
ratios_to_test = np.arange(0.5, 2.1, 0.5)

# Store results in dictionaries for easier plotting later
results_fixed_density = defaultdict(dict)
results_fixed_snakes = defaultdict(dict)
results_fixed_ladders = defaultdict(dict)

In [ ]:
# Cell 8: Run simulations for fixed density of snakes and ladders (Modified to pass lengths to run_simulations)
print("Running simulations for fixed density (Ns/Nl = 1.0)...")
for board_size in board_sizes:
    tiles = board_size * board_size
    num_entities = int(0.1 * tiles) # 0.1 density
    num_snakes = num_entities
    num_ladders = num_entities

    print(f"Simulating board size: {board_size}x{board_size} with {num_snakes} snakes and {num_ladders} ladders...")
    simulation_results = run_simulations(board_size, num_simulations, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
    results_fixed_density[board_size] = simulation_results

    # Generate and save board layout for Ns/Nl = 1.0 (fixed density case)
    if board_size == 10: # Example board layout for 10x10, can change board_size value as needed.
        board_snakes_example, board_ladders_example = create_board(board_size, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
        generate_board_layout_visual(board_size, board_snakes_example, board_ladders_example, filename=f"board_layout_{board_size}x{board_size}_density_1.0.png")

print("Fixed density simulations complete.")

In [ ]:
# Cell 9: Run simulations for varying Ns/Nl ratio (Fixed Snakes) (Modified to pass lengths to run_simulations)
print("\nRunning simulations for varying Ns/Nl (Fixed Snakes)...")
fixed_snakes_10x10 = int(0.1 * 10 * 10) # Fixed number of snakes based on 10x10 board density
for board_size in board_sizes:
    for ratio in ratios_to_test:
        num_snakes = fixed_snakes_10x10 # Keep snakes fixed
        num_ladders = int(num_snakes / ratio) if ratio != 0 else 0 # Calculate ladders based on ratio

        if num_ladders < 0: # Handle cases with negative ladders (shouldn't occur with ratios >= 0.5 but for robustness)
            num_ladders = 0

        print(f"Simulating board size: {board_size}x{board_size}, Ns/Nl Ratio: {ratio}, Snakes: {num_snakes}, Ladders: {num_ladders}")
        simulation_results = run_simulations(board_size, num_simulations, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
        results_fixed_snakes[(board_size, ratio)] = simulation_results

        # Generate and save board layout for Ns/Nl = 1.0 (Fixed Snakes example for 10x10)
        if board_size == 10 and ratio == 1.0:
            board_snakes_example_ratio_1, board_ladders_example_ratio_1 = create_board(board_size, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
            generate_board_layout_visual(board_size, board_snakes_example_ratio_1, board_ladders_example_ratio_1, filename=f"board_layout_{board_size}x{board_size}_fixed_snakes_ratio_1.0.png")


print("Fixed Snakes simulations complete.")

In [ ]:
# Cell 10: Run simulations for varying Ns/Nl ratio (Fixed Ladders) (Modified to pass lengths to run_simulations)
print("\nRunning simulations for varying Ns/Nl (Fixed Ladders)...")
fixed_ladders_10x10 = int(0.1 * 10 * 10) # Fixed number of ladders based on 10x10 board density
for board_size in board_sizes:
    for ratio in ratios_to_test:
        num_ladders = fixed_ladders_10x10 # Keep ladders fixed
        num_snakes = int(num_ladders * ratio) # Calculate snakes based on ratio

        print(f"Simulating board size: {board_size}x{board_size}, Ns/Nl Ratio: {ratio}, Snakes: {num_snakes}, Ladders: {num_ladders}")
        simulation_results = run_simulations(board_size, num_simulations, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
        results_fixed_ladders[(board_size, ratio)] = simulation_results

        # Generate and save board layout for Ns/Nl = 1.0 (Fixed Ladders example for 10x10)
        if board_size == 10 and ratio == 1.0:
            board_snakes_example_ratio_2, board_ladders_example_ratio_2 = create_board(board_size, num_snakes, num_ladders, snake_ladder_length, snake_ladder_length) # Pass lengths
            generate_board_layout_visual(board_size, board_snakes_example_ratio_2, board_ladders_example_ratio_2, filename=f"board_layout_{board_size}x{board_size}_fixed_ladders_ratio_1.0.png")


print("Fixed Ladders simulations complete.")

In [65]:
# Cell A: ISOLATED BOARD CREATION TEST
board_size_test = 8 # Test with 8x8 board
num_entities_test = 6 # Test with 6 snakes and 6 ladders (for 8x8 board)
snake_length_test = 10
ladder_length_test = 10

print(f"--- ISOLATED BOARD CREATION TEST: {board_size_test}x{board_size_test} with {num_entities_test} entities ---")
board_snakes_test, board_ladders_test = create_board(board_size_test, num_entities_test, num_entities_test, snake_length_test, ladder_length_test)

if board_snakes_test and board_ladders_test:
    print("\nBoard creation SUCCESSFUL!")
    print("\nSnakes:", board_snakes_test)
    print("\nLadders:", board_ladders_test)
else:
    print("\nBoard creation FAILED!")

--- ISOLATED BOARD CREATION TEST: 8x8 with 6 entities ---
Attempting to place snake 1/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 3578, end: 3568
    _check_overlap returning True
    snake placement VALID - start: 3578, end: 3568
Snake placed successfully. Snakes placed: 1/6
Attempting to place snake 2/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 2185, end: 2175
    _check_overlap returning True
    snake placement VALID - start: 2185, end: 2175
Snake placed successfully. Snakes placed: 2/6
Attempting to place snake 3/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 1229, end: 1219
    _check_overlap returning True
    snake placement VALID - start: 1229, end: 1219
Snake placed successfully. Snakes placed: 3/6
Attempting to place snake 4/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 2781, end: 2771
    _check_overlap returning 

In [66]:
# Cell B: ISOLATED GAME SIMULATION TEST
board_size_sim_test = 8 # Board size for simulation test

# MANUALLY DEFINED board (for testing simulation logic - NO RANDOM BOARD GENERATION HERE)
board_snakes_sim_test = {16: 4, 25: 5, 36: 6} # Example snakes
board_ladders_sim_test = {9: 27, 18: 40, 30: 50} # Example ladders

print(f"--- ISOLATED GAME SIMULATION TEST: {board_size_sim_test}x{board_size_sim_test} ---")
print("\nUsing MANUAL BOARD:")
print("\nSnakes:", board_snakes_sim_test)
print("\nLadders:", board_ladders_sim_test)

turns_sim_test = simulate_game(board_size_sim_test, board_snakes_sim_test, board_ladders_sim_test)
print(f"\nGame simulation COMPLETED in {turns_sim_test} turns.")

--- ISOLATED GAME SIMULATION TEST: 8x8 ---

Using MANUAL BOARD:

Snakes: {16: 4, 25: 5, 36: 6}

Ladders: {9: 27, 18: 40, 30: 50}

Game simulation COMPLETED in 9 turns.


In [67]:
# Cell C: SIMPLIFIED SIMULATION LOOP TEST (ONE BOARD SIZE, FEW SIMULATIONS)
board_size_loop_test = 8 # Test with 8x8 board
num_entities_loop_test = 6 # Test with 6 snakes and 6 ladders
num_simulations_loop_test = 10 # Reduced simulations for testing

print(f"--- SIMPLIFIED SIMULATION LOOP TEST: {board_size_loop_test}x{board_size_loop_test}, {num_simulations_loop_test} simulations ---")

simulation_results_loop_test = run_simulations(board_size_loop_test, num_simulations_loop_test, num_entities_loop_test, num_entities_loop_test, snake_ladder_length, snake_ladder_length)

print("\nSimulation loop TEST COMPLETED.")
print("\nAverage turns:", simulation_results_loop_test["average_turns"])
print("\nWin probabilities:", {k: v for k, v in simulation_results_loop_test.items() if k.startswith("prob_win")})

--- SIMPLIFIED SIMULATION LOOP TEST: 8x8, 10 simulations ---
Attempting to place snake 1/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 1702, end: 1692
    _check_overlap returning True
    snake placement VALID - start: 1702, end: 1692
Snake placed successfully. Snakes placed: 1/6
Attempting to place snake 2/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 2337, end: 2327
    _check_overlap returning True
    snake placement VALID - start: 2337, end: 2327
Snake placed successfully. Snakes placed: 2/6
Attempting to place snake 3/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 2522, end: 2512
    _check_overlap returning True
    snake placement VALID - start: 2522, end: 2512
Snake placed successfully. Snakes placed: 3/6
Attempting to place snake 4/6
  generate_endpoints attempt 1 for snake
    _check_overlap called for snake start: 3624, end: 3614
    _check_overlap returni

In [ ]:
# Cell 11: Function to plot Average Game Time vs Ns/Nl Ratio with Error Bars
def plot_avg_game_time_ratio_error_bars(results_data, fixed_entity_type, filename="avg_game_time_ratio.png"):
    """Plots Average Game Time vs Ns/Nl Ratio with error bars.

    Args:
        results_data (dict): Results dictionary (results_fixed_snakes or results_fixed_ladders).
        fixed_entity_type (str): "Snakes" or "Ladders" to indicate which entity was fixed.
        filename (str, optional): Filename to save the plot. Defaults to "avg_game_time_ratio.png".
    """
    plt.figure(figsize=(10, 6))

    ratios = sorted(list(set([ratio for _, ratio in results_data.keys()]))) # Get sorted ratios
    board_sizes_in_results = sorted(list(set([size for size, _ in results_data.keys()]))) # Get sorted board sizes

    for board_size in board_sizes_in_results:
        avg_times = []
        std_errors = [] # For error bars
        for ratio in ratios:
            if (board_size, ratio) in results_data:
                results = results_data[(board_size, ratio)]
                avg_times.append(results["average_turns"])

                # Calculate standard error of the mean for error bars
                std_dev_turns = np.std(results["all_turns"])
                std_error_turns = std_dev_turns / np.sqrt(len(results["all_turns"])) if len(results["all_turns"]) > 1 else 0 #Avoid division by zero if only one simulation
                std_errors.append(std_error_turns)
            else:
                avg_times.append(np.nan) # Handle missing data points with NaN
                std_errors.append(np.nan)


        plt.errorbar(ratios, avg_times, yerr=std_errors, label=f"Board Size {board_size}x{board_size}", capsize=5, marker='o', linestyle='-')


    plt.xlabel("Ns/Nl Ratio")
    plt.ylabel("Average Game Time (Turns)")
    plt.title(f"Average Game Time vs Ns/Nl Ratio (Fixed {fixed_entity_type})")
    plt.xticks(ratios) # Ensure x-axis ticks are at each ratio value
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    plt.close()
    print(f"Average Game Time vs Ns/Nl Ratio plot (with error bars) saved to {filename}")

In [ ]:
# Cell 12: Function to plot Average Game Time vs Ns/Nl Ratio Aggregated
def plot_aggregated_avg_game_time_ratio(results_data, fixed_entity_type, filename="aggregated_avg_game_time_ratio.png"):
    """Plots Aggregated Average Game Time vs Ns/Nl Ratio (aggregated across board sizes).

    Args:
        results_data (dict): Results dictionary (results_fixed_snakes or results_fixed_ladders).
        fixed_entity_type (str): "Snakes" or "Ladders" to indicate which entity was fixed.
        filename (str, optional): Filename to save the plot. Defaults to "aggregated_avg_game_time_ratio.png".
    """
    plt.figure(figsize=(10, 6))

    ratios = sorted(list(set([ratio for _, ratio in results_data.keys()]))) # Get sorted ratios
    aggregated_avg_times = []
    aggregated_std_errors = [] # For error bars

    for ratio in ratios:
        all_board_sizes_avg_times = []
        all_board_sizes_all_turns = []

        for board_size, current_ratio in results_data.keys():
            if current_ratio == ratio:
                results = results_data[(board_size, ratio)]
                all_board_sizes_avg_times.append(results["average_turns"])
                all_board_sizes_all_turns.extend(results["all_turns"]) # Collect all turns for error bar calculation

        aggregated_avg_times.append(np.mean(all_board_sizes_avg_times))

        # Standard error calculation for aggregated data
        aggregated_std_dev_turns = np.std(all_board_sizes_all_turns)
        aggregated_std_error_turns = aggregated_std_dev_turns / np.sqrt(len(all_board_sizes_all_turns)) if len(all_board_sizes_all_turns) > 1 else 0
        aggregated_std_errors.append(aggregated_std_error_turns)


    plt.errorbar(ratios, aggregated_avg_times, yerr=aggregated_std_errors, capsize=5, marker='o', linestyle='-', label="Aggregated Average Game Time")

    plt.xlabel("Ns/Nl Ratio")
    plt.ylabel("Aggregated Average Game Time (Turns)")
    plt.title(f"Aggregated Average Game Time vs Ns/Nl Ratio (Fixed {fixed_entity_type}, All Board Sizes)")
    plt.xticks(ratios)
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    plt.close()
    print(f"Aggregated Average Game Time vs Ns/Nl Ratio plot (with error bars) saved to {filename}")

In [ ]:
# Cell 13: Function to plot Board Size vs Win Probability
def plot_win_probability_board_size(results_data, prob_type, ratio_value=None, filename="win_probability_board_size.png"):
    """Plots Board Size vs Probability of Winning within N/turns.

    Args:
        results_data (dict): Results dictionary (results_fixed_density, results_fixed_snakes, or results_fixed_ladders).
        prob_type (str): "half", "third", or "quarter" to specify win probability type (N/2, N/3, N/4 turns).
        ratio_value (float, optional): Ns/Nl ratio value to filter data for (for ratio-specific plots). Defaults to None for fixed density plot.
        filename (str, optional): Filename to save the plot. Defaults to "win_probability_board_size.png".
    """
    plt.figure(figsize=(10, 6))
    board_sizes_in_results = sorted(list(set([size for size in results_data.keys()] if ratio_value is None else [size for size, ratio in results_data.keys() if ratio == ratio_value]))) # Sorted board sizes, filter by ratio if ratio_value is given

    prob_key = f"prob_win_{prob_type}" # Dynamically create probability key

    board_sizes_to_plot = []
    win_probabilities = []

    for board_size in board_sizes_in_results:
        if ratio_value is None: # For fixed density results
            if board_size in results_data:
                win_prob = results_data[board_size][prob_key]
                board_sizes_to_plot.append(board_size)
                win_probabilities.append(win_prob)
        elif (board_size, ratio_value) in results_data: # For ratio-specific results
             win_prob = results_data[(board_size, ratio_value)][prob_key]
             board_sizes_to_plot.append(board_size)
             win_probabilities.append(win_prob)


    plt.plot(board_sizes_to_plot, win_probabilities, marker='o', linestyle='-')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel(f"Probability of Winning within N/{prob_type[4:]} Turns") # Dynamically label y-axis
    title_ratio_part = f" (Ns/Nl = {ratio_value})" if ratio_value is not None else "" # Add ratio to title if applicable
    plt.title(f"Board Size vs Probability of Winning within N/{prob_type[4:]} Turns{title_ratio_part}")
    plt.xticks(board_sizes_to_plot) # Ensure x-axis ticks at each board size
    plt.grid(True)
    plt.savefig(filename)
    plt.close()
    print(f"Board Size vs Probability of Winning plot saved to {filename}")

In [ ]:
# Cell 14: Function to plot Board Size vs Average Number of Turns
def plot_avg_turns_board_size(results_data, ratio_value=None, filename="avg_turns_board_size.png"):
    """Plots Board Size vs Average Number of Turns.

    Args:
        results_data (dict): Results dictionary (results_fixed_density, results_fixed_snakes, or results_fixed_ladders).
        ratio_value (float, optional): Ns/Nl ratio value to filter data for (for ratio-specific plots). Defaults to None for fixed density plot.
        filename (str, optional): Filename to save the plot. Defaults to "avg_turns_board_size.png".
    """
    plt.figure(figsize=(10, 6))
    board_sizes_in_results = sorted(list(set([size for size in results_data.keys()] if ratio_value is None else [size for size, ratio in results_data.keys() if ratio == ratio_value]))) # Sorted board sizes, filter by ratio if ratio_value is given


    board_sizes_to_plot = []
    avg_turns_values = []

    for board_size in board_sizes_in_results:
        if ratio_value is None: # For fixed density results
            if board_size in results_data:
                avg_turns = results_data[board_size]["average_turns"]
                board_sizes_to_plot.append(board_size)
                avg_turns_values.append(avg_turns)
        elif (board_size, ratio_value) in results_data: # For ratio-specific results
             avg_turns = results_data[(board_size, ratio_value)]["average_turns"]
             board_sizes_to_plot.append(board_size)
             avg_turns_values.append(avg_turns)


    plt.plot(board_sizes_to_plot, avg_turns_values, marker='o', linestyle='-')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel("Average Number of Turns")
    title_ratio_part = f" (Ns/Nl = {ratio_value})" if ratio_value is not None else "" # Add ratio to title if applicable
    plt.title(f"Board Size vs Average Number of Turns{title_ratio_part}")
    plt.xticks(board_sizes_to_plot) # Ensure x-axis ticks at each board size
    plt.grid(True)
    plt.savefig(filename)
    plt.close()
    print(f"Board Size vs Average Number of Turns plot saved to {filename}")

In [ ]:
# Cell 15: Generate plots
# 1. Board Layouts (already generated during simulations in cells 8, 9, 10 and saved as PNG files)

# 2. Average Game Time vs Ns/Nl Ratio Plots (with error bars)
plot_avg_game_time_ratio_error_bars(results_fixed_snakes, "Snakes", filename="avg_game_time_ratio_fixed_snakes_error_bars.png")
plot_avg_game_time_ratio_error_bars(results_fixed_ladders, "Ladders", filename="avg_game_time_ratio_fixed_ladders_error_bars.png")

# 3. Aggregated Average Game Time vs Ns/Nl Ratio Plots (with error bars)
plot_aggregated_avg_game_time_ratio(results_fixed_snakes, "Snakes", filename="aggregated_avg_game_time_ratio_fixed_snakes_error_bars.png")
plot_aggregated_avg_game_time_ratio(results_fixed_ladders, "Ladders", filename="aggregated_avg_game_time_ratio_fixed_ladders_error_bars.png")

# 4. Board Size vs Probability of Winning Plots
for prob_type in ["half", "third", "quarter"]:
    plot_win_probability_board_size(results_fixed_density, prob_type, filename=f"board_size_vs_win_prob_density_1.0_{prob_type}.png") # For density 1.0
    for ratio in ratios_to_test:
        plot_win_probability_board_size(results_fixed_snakes, prob_type, ratio, filename=f"board_size_vs_win_prob_fixed_snakes_ratio_{ratio}_{prob_type}.png") # For fixed snakes, varying ratios
        plot_win_probability_board_size(results_fixed_ladders, prob_type, ratio, filename=f"board_size_vs_win_prob_fixed_ladders_ratio_{ratio}_{prob_type}.png") # For fixed ladders, varying ratios

# 5. Board Size vs Average Number of Turns Plots
plot_avg_turns_board_size(results_fixed_density, filename="board_size_vs_avg_turns_density_1.0.png") # For density 1.0
for ratio in ratios_to_test:
    plot_avg_turns_board_size(results_fixed_snakes, ratio, filename=f"board_size_vs_avg_turns_fixed_snakes_ratio_{ratio}.png") # For fixed snakes, varying ratios
    plot_avg_turns_board_size(results_fixed_ladders, ratio, filename=f"board_size_vs_avg_turns_fixed_ladders_ratio_{ratio}.png") # For fixed ladders, varying ratios

print("All plots generated and saved as PNG files.")